# 1. Import required libraries

In [221]:
# Import required dependencies
import boto3
import pandas as pd
import numpy as np


# 2. Read data

In [222]:
# Readind data
bucket_name = 'finance-factoring-ml-dev'

df = pd.read_csv('s3://{}/data/all.csv'.format(bucket_name))
df.head()

,Fecha,RFCCliente,RFCPagador,Moneda,FechaVencimiento,Monto,DepositoInicial,Intereses,FechaPago
0,1918.0,DDI140206JB6,0,1,1953.0,38879.48,33047.56,2351.94,1939.0
1,1634.0,GOSJ630117VA4,1,0,1654.0,6658.40,5659.64,109.45,1638.0
2,1583.0,ECM190924RS5,2,1,1551.0,791.12,791.12,29.26,1551.0
3,2282.0,DIM9811232SA,3,0,2225.0,33048.00,28090.80,814.88,2226.0
4,2110.0,SNO200526BV7,4,0,2170.0,424634.82,360939.60,10615.87,2166.0


# 3. Data Engineering

In [223]:
# Remove unnecesary columns
df.drop("RFCPagador", axis=1, inplace=True)
df.drop("Moneda", axis=1, inplace=True)
df.drop("DepositoInicial", axis=1, inplace=True)
df.drop("Intereses", axis=1, inplace=True)
df.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,FechaPago
0,1918.0,DDI140206JB6,1953.0,38879.48,1939.0
1,1634.0,GOSJ630117VA4,1654.0,6658.40,1638.0
2,1583.0,ECM190924RS5,1551.0,791.12,1551.0
3,2282.0,DIM9811232SA,2225.0,33048.00,2226.0
4,2110.0,SNO200526BV7,2170.0,424634.82,2166.0


In [224]:
# Create the input DataFrame
input = [[1918.0, 'DDI140206JB6', 1953.0, 38879.48]]
columns = ['Fecha', 'RFCCliente', 'FechaVencimiento', 'Monto']

df_input = pd.DataFrame(input, columns=columns)
df_input

,Fecha,RFCCliente,FechaVencimiento,Monto
0,1918.0,DDI140206JB6,1953.0,38879.48


In [228]:
# Build feature Number of total paid invoices
def calc_number_total_paid_invoices(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha)]
    
    return len(df_filtered.index)

df_input['NumberTotalPaidInvoices'] = df_input.apply(calc_number_total_paid_invoices, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297


In [189]:
# Build feature Number of invoices that were paid late
def calc_number_invoices_paid_late(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha)  & (df.FechaPago > df.FechaVencimiento)]
    
    return len(df_filtered.index)

df_input['NumberInvoicesPaidLate'] = df_input.apply(calc_number_invoices_paid_late, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67


In [190]:
# Build feature Ratio of paid invoices that were late
def calc_ratio_invoices_paid_late(row):
    if row.NumberTotalPaidInvoices == 0:
        return 0
    else:
        return row.NumberInvoicesPaidLate / row.NumberTotalPaidInvoices

df_input['RatioInvoicesPaidLate'] = df_input.apply(calc_ratio_invoices_paid_late, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589


In [191]:
# Build feature Sum of the base amount of total paid invoices
def calc_sum_amount_total_paid_invoices(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha)]
    
    return df_filtered.Monto.sum()

df_input['SumAmountTotalPaidInvoices'] = df_input.apply(calc_sum_amount_total_paid_invoices, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31


In [192]:
# Build feature Sum of the base amount of invoices that were paid late
def calc_sum_amount_invoices_paid_late(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > df.FechaVencimiento)]
    
    return df_filtered.Monto.sum()

df_input['SumAmountInvoicesPaidLate'] = df_input.apply(calc_sum_amount_invoices_paid_late, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3


In [193]:
# Build feature Ratio of sum of paid base amount that were late
def calc_ratio_sum_amount_invoices_paid_late(row):
    if row.SumAmountTotalPaidInvoices == 0:
        return 0
    else:
        return row.SumAmountInvoicesPaidLate / row.SumAmountTotalPaidInvoices

df_input['RatioSumAmountInvoicesPaidLate'] = df_input.apply(calc_ratio_sum_amount_invoices_paid_late, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978


In [194]:
# Build feature Average days late of paid invoices being late.
def calc_avg_days_late_invoices_paid_late(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > df.FechaVencimiento)]
  
    df_filtered['DaysLate'] = df_filtered['FechaPago'] - df_filtered['FechaVencimiento']
    
    if len(df_filtered.index) == 0:
        return 0
    else:
        return df_filtered.DaysLate.mean()

df_input['AvgDaysLateInvoicesPaidLate'] = df_input.apply(calc_avg_days_late_invoices_paid_late, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493


In [195]:
# Build feature Number of total outstanding invoices
def calc_number_total_outstanding_invoices(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > row.Fecha)]
  
    return len(df_filtered.index)

df_input['NumberTotalOutstandingInvoices'] = df_input.apply(calc_number_total_outstanding_invoices, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493,20


In [196]:
# Build feature Number of outstanding invoices that were already late
def calc_number_outstanding_late_invoices(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > df.FechaVencimiento) & (df.FechaPago > row.Fecha)]
  
    return len(df_filtered.index)

df_input['NumberOutstandingLateInvoices'] = df_input.apply(calc_number_outstanding_late_invoices, axis=1)
df_input.head()


,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493,20,10


In [197]:
# Build feature Ratio of outstanding invoices that were late
def calc_ratio_outstanding_late_invoices(row):
    if row.NumberTotalOutstandingInvoices == 0:
        return 0
    else:
        return row.NumberOutstandingLateInvoices / row.NumberTotalOutstandingInvoices

df_input['RatioOutstandingLateInvoices'] = df_input.apply(calc_ratio_outstanding_late_invoices, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices,RatioOutstandingLateInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493,20,10,0.5


In [198]:
# Build feature Sum of the base amount of total outstanding invoices
def calc_sum_amount_total_outstanding_invoices(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > row.Fecha)]
  
    return df_filtered.Monto.sum()

df_input['SumAmountTotalOutstandingInvoices'] = df_input.apply(calc_sum_amount_total_outstanding_invoices, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices,RatioOutstandingLateInvoices,SumAmountTotalOutstandingInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493,20,10,0.5,510099.33


In [199]:
# Build feature Sum of the base amount of outstanding invoices that were late
def calc_sum_amount_outstanding_late_invoices(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > row.Fecha) & (df.FechaPago > df.FechaVencimiento)]
  
    return df_filtered.Monto.sum()

df_input['SumAmountOutstandingLateInvoices'] = df_input.apply(calc_sum_amount_outstanding_late_invoices, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices,RatioOutstandingLateInvoices,SumAmountTotalOutstandingInvoices,SumAmountOutstandingLateInvoices
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493,20,10,0.5,510099.33,249381.4


In [200]:
# Build feature Ratio of sum of outstanding base amount that were late
def calc_ratio_sum_amount_outstanding_late_invoice(row):
    if row.SumAmountTotalOutstandingInvoices == 0:
        return 0
    else:
        return row.SumAmountOutstandingLateInvoices / row.SumAmountTotalOutstandingInvoices

df_input['RatioSumAmountOutstandingLateInvoice'] = df_input.apply(calc_ratio_sum_amount_outstanding_late_invoice, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices,RatioOutstandingLateInvoices,SumAmountTotalOutstandingInvoices,SumAmountOutstandingLateInvoices,RatioSumAmountOutstandingLateInvoice
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,15.701493,20,10,0.5,510099.33,249381.4,0.488888


In [201]:
# Build feature Average days late of outstanding invoices being late.
def calc_avg_days_late_invoices_paid_late(row):
    df_filtered = df[(df.RFCCliente == row.RFCCliente) & (df.Fecha < row.Fecha) & (df.FechaPago > df.FechaVencimiento) & (df.FechaPago > row.Fecha)]
  
    df_filtered['DaysLate'] = df_filtered['FechaPago'] - df_filtered['FechaVencimiento']
    
    if len(df_filtered.index) == 0:
        return 0
    else:
        return df_filtered.DaysLate.mean()

df_input['AvgDaysLateInvoicesPaidLate'] = df_input.apply(calc_avg_days_late_invoices_paid_late, axis=1)
df_input.head()

,Fecha,RFCCliente,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices,RatioOutstandingLateInvoices,SumAmountTotalOutstandingInvoices,SumAmountOutstandingLateInvoices,RatioSumAmountOutstandingLateInvoice
0,1918.0,DDI140206JB6,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,19.4,20,10,0.5,510099.33,249381.4,0.488888


## 3.0 Clean up

In [202]:
# Remove RFC Cliente feature because it's no longer needed
df_input.drop("RFCCliente", axis=1, inplace=True)
df_input.head()

,Fecha,FechaVencimiento,Monto,NumberTotalPaidInvoices,NumberInvoicesPaidLate,RatioInvoicesPaidLate,SumAmountTotalPaidInvoices,SumAmountInvoicesPaidLate,RatioSumAmountInvoicesPaidLate,AvgDaysLateInvoicesPaidLate,NumberTotalOutstandingInvoices,NumberOutstandingLateInvoices,RatioOutstandingLateInvoices,SumAmountTotalOutstandingInvoices,SumAmountOutstandingLateInvoices,RatioSumAmountOutstandingLateInvoice
0,1918.0,1953.0,38879.48,297,67,0.225589,6888834.31,1687615.3,0.244978,19.4,20,10,0.5,510099.33,249381.4,0.488888


In [205]:
input_data = df_input.values.tolist()
print(input_data)

[[1918.0, 1953.0, 38879.48, 297.0, 67.0, 0.2255892255892256, 6888834.3100000005, 1687615.3, 0.24497835541641877, 19.4, 20.0, 10.0, 0.5, 510099.3300000001, 249381.4, 0.4888879191431205]]
